# Getting COT data

### Imports

In [78]:
import pandas as pd
from index import cot_all

In [79]:
test_df = pd.read_csv 

### COT from 2017 till now

In [80]:
all_df = cot_all()

used cached file legacy_fut.csv under the directory: /home/mod7ex/cot-py/data


### All COT data

In [81]:
all_df = all_df.rename(columns={'As of Date in Form YYYY-MM-DD': 'Date'})

all_df['Date'] = pd.to_datetime(all_df['Date'])

all_df.set_index('Date', inplace=True)

all_df.sort_index(inplace=True)

all_df.head()

Market and Exchange Names  \
Date                                                            
1986-01-15  CRUDE OIL, LIGHT 'SWEET' - NEW YORK MERCANTILE...   
1986-01-15                      CORN - CHICAGO BOARD OF TRADE   
1986-01-15          LIVE CATTLE - CHICAGO MERCANTILE EXCHANGE   
1986-01-15              SOYBEAN MEAL - CHICAGO BOARD OF TRADE   
1986-01-15  PROPANE GAS - PETROLEUM ASSOC OF N Y COTTON EXCH.   

            As of Date in Form YYMMDD CFTC Contract Market Code  \
Date                                                              
1986-01-15                     860115                    067651   
1986-01-15                     860115                    002601   
1986-01-15                     860115                    057642   
1986-01-15                     860115                    026603   
1986-01-15                     860115                    066752   

           CFTC Market Code in Initials  CFTC Region Code  \
Date                                                        
1986-01-15                         NYME                 1   
1986-01-15                          CBT                 0   
1986-01-15                          CME                 0   
1986-01-15                          CBT                 0   
1986-01-15                         PANY                 1   

            CFTC Commodity Code  Open Interest (All)  \
Date                                                   
1986-01-15                   67                74334   
1986-01-15                    2               601935   
1986-01-15                   57                58475   
1986-01-15                   26                46028   
1986-01-15                   66                  303   

            Noncommercial Positions-Long (All)  \
Date                                             
1986-01-15                                2560   
1986-01-15                               45615   
1986-01-15                                3475   
1986-01-15                                7292   
1986-01-15                                   0   

            Noncommercial Positions-Short (All)  \
Date                                              
1986-01-15                                11594   
1986-01-15                                16565   
1986-01-15                                 4915   
1986-01-15                                  689   
1986-01-15                                    0   

            Noncommercial Positions-Spreading (All)  ...  \
Date                                                 ...   
1986-01-15                                     5317  ...   
1986-01-15                                    28015  ...   
1986-01-15                                     3842  ...   
1986-01-15                                     1780  ...   
1986-01-15                                        0  ...   

            Concentration-Gross LT =8 TDR-Long (Other)  \
Date                                                     
1986-01-15                                         0.0   
1986-01-15                                        46.7   
1986-01-15                                         0.0   
1986-01-15                                        27.2   
1986-01-15                                         0.0   

            Concentration-Gross LT =8 TDR-Short(Other)  \
Date                                                     
1986-01-15                                        69.8   
1986-01-15                                        21.6   
1986-01-15                                         0.0   
1986-01-15                                        47.5   
1986-01-15                                         0.0   

            Concentration-Net LT =4 TDR-Long (Other)  \
Date                                                   
1986-01-15                                       0.0   
1986-01-15                                      35.3   
1986-01-15                                       0.0   
1986-01-15                                      16.6   
1986-01-1

## Filter the asset

In [ ]:
ASSET_CODE = '088691'

asset_df = all_df[all_df['CFTC Contract Market Code'] == ASSET_CODE]
asset_df = asset_df[asset_df.index > '2020']

In [ ]:
# all_df[(all_df['Open Interest (Other)']!= 0) & (all_df['CFTC Contract Market Code'] == ASSET_CODE)]

## COT index

In [85]:
commercials_df = asset_df[['Commercial Positions-Long (All)', 'Commercial Positions-Short (All)']].copy()

commercials_df.columns = ['Long', 'Short']

commercials_df['Net Position'] = commercials_df['Long'] - commercials_df['Short']

commercials_df

Long   Short  Net Position
Date                                    
2020-01-07  202452  558533       -356081
2020-01-14  221358  571177       -349819
2020-01-21  212445  564348       -351903
2020-01-28  192300  553114       -360814
2020-02-04  176786  508473       -331687
...            ...     ...           ...
2025-06-17   71684  309411       -237727
2025-06-24   73323  303883       -230560
2025-07-01   68820  304889       -236069
2025-07-08   71875  310229       -238354
2025-07-15   75989  326677       -250688

[289 rows x 3 columns]

## COT Index Formula

The Commitments of Traders (COT) Index is calculated as:

$$
\text{COT Index} = \left( \frac{P_{\text{current}} - P_{\min}}{P_{\max} - P_{\min}} \right) \times 100
$$

Where:

$ P_{\text{current}} $: Current Net Position <br>
$ P_{\min} $: Lowest Net Position in the past  N  period <br>
$ P_{\max} $: Highest Net Position in the past  N  periods <br>
$ N $: Number of periods used in the lookback window


| Strategy                         | Recommended COT Index Period | Notes                                                           |
| -------------------------------- | ---------------------------- | --------------------------------------------------------------- |
| **Swing Trading**                | **26 weeks** (6 months)      | Most popular. Balances short-term moves and sentiment extremes. |
| **Position Trading / Long-Term** | **52 weeks** (1 year)        | Captures major sentiment extremes and long-term reversals.      |
| **Short-Term Trading**           | **13 weeks** (3 months)      | More sensitive to recent changes, but can be noisy.             |
| **Contrarian Approach**          | Use 26–52 weeks              | To spot positioning at sentiment extremes (close to 0 or 100).  |


In [86]:
window = 26 # Rolling window for the last N weeks

commercials_df['Max_Net_Position_Last_N'] = commercials_df['Net Position'].rolling(window=window).max()

commercials_df['Min_Net_Position_Last_N'] = commercials_df['Net Position'].rolling(window=window).min()

commercials_df.dropna(inplace=True)

commercials_df['Index'] = 100 * (commercials_df['Net Position'] - commercials_df['Min_Net_Position_Last_N'])/(commercials_df['Max_Net_Position_Last_N'] - commercials_df['Min_Net_Position_Last_N'])

commercials_df

Long   Short  Net Position  Max_Net_Position_Last_N  \
Date                                                                
2020-06-30  116697  418160       -301463                -243457.0   
2020-07-07  118424  421016       -302592                -243457.0   
2020-07-14  125909  427927       -302018                -243457.0   
2020-07-21  129690  433549       -303859                -243457.0   
2020-07-28  139803  418180       -278377                -243457.0   
...            ...     ...           ...                      ...   
2025-06-17   71684  309411       -237727                -190761.0   
2025-06-24   73323  303883       -230560                -190761.0   
2025-07-01   68820  304889       -236069                -190761.0   
2025-07-08   71875  310229       -238354                -190761.0   
2025-07-15   75989  326677       -250688                -190761.0   

            Min_Net_Position_Last_N      Index  
Date                                            
2020-06-30                -385612.0  59.195245  
2020-07-07                -385612.0  58.401041  
2020-07-14                -385612.0  58.804826  
2020-07-21                -385612.0  57.509760  
2020-07-28                -385612.0  75.435264  
...                             ...        ...  
2025-06-17                -328175.0  65.821532  
2025-06-24                -328175.0  71.037158  
2025-07-01                -328175.0  67.028105  
2025-07-08                -328175.0  65.365247  
2025-07-15                -328175.0  56.389451  

[264 rows x 6 columns]

### Plotting COT index

In [87]:
import plotly.express as px

fig = px.line(commercials_df.reset_index(), x='Date', y='Index', title='COT index', markers=True, template='plotly_dark')

# Add green shaded area (e.g. for high index range)
fig.add_shape(
    type='rect',
    xref='paper',  # entire x-axis
    yref='y',
    x0=0,
    x1=1,
    y0=80,
    y1=100,
    fillcolor='green',
    opacity=0.2,
    line_width=0,
    layer='below'
)

# Add red shaded area (e.g. for low index range)
fig.add_shape(
    type='rect',
    xref='paper',
    yref='y',
    x0=0,
    x1=1,
    y0=0,
    y1=20,
    fillcolor='red',
    opacity=0.2,
    line_width=0,
    layer='below'
)

fig.write_html("plots/commercials_index.html")
fig.show()


## Open Interest

In [88]:
oi_df = asset_df[['Open Interest (All)']].copy()

oi_df.columns = ['OI']

oi_df

OI
Date              
2020-01-07  785857
2020-01-14  796883
2020-01-21  793829
2020-01-28  715539
2020-02-04  654572
...            ...
2025-06-17  441214
2025-06-24  434958
2025-07-01  437662
2025-07-08  443144
2025-07-15  448531

[289 rows x 1 columns]

### plotting

In [89]:
import plotly.express as px

fig = px.line(oi_df.reset_index(), x='Date', y='OI', title='Open Interest', markers=True, template='plotly_dark')


fig.write_html("plots/open_interest.html")
fig.show()

## Net positions

> Commercial (Long) = Producer/Merchant/Processor/User(Long) + Swap Dealers(Long) + Swap Dealers(Spreading)

> Commercial (Short) = Producer/Merchant/Processor/User(Short) + Swap Dealers(Short) + Swap Dealers(Spreading)

> Non-Commercial (Long) = Managed Money(Long) + Other Reportables(Long) 

> Non-Commercial (Short) = Managed Money(Short) + Other Reportables(Short) 

> Non-Commercial (Spreading) = Managed Money(Spreading) + Other Reportables(Spreading) 

In [90]:
net_positions_df = pd.DataFrame()

net_positions_df['Commercials'] = commercials_df['Net Position']
net_positions_df['Large speculators'] = asset_df['Noncommercial Positions-Long (All)'] - asset_df['Noncommercial Positions-Short (All)']
net_positions_df['Small Traders'] = asset_df['Nonreportable Positions-Long (All)'] - asset_df['Nonreportable Positions-Short (All)']

net_positions_df

Commercials  Large speculators  Small Traders
Date                                                     
2020-06-30      -301463             266670          34793
2020-07-07      -302592             267358          35234
2020-07-14      -302018             262428          39590
2020-07-21      -303859             266436          37423
2020-07-28      -278377             236801          41576
...                 ...                ...            ...
2025-06-17      -237727             200648          37079
2025-06-24      -230560             195004          35556
2025-07-01      -236069             201980          34089
2025-07-08      -238354             202968          35386
2025-07-15      -250688             213115          37573

[264 rows x 3 columns]

### Plotting

In [91]:
import plotly.graph_objects as go

fig = go.Figure()

for col in net_positions_df.columns:
    fig.add_trace(go.Scatter(x=net_positions_df.index, y=net_positions_df[col], mode='lines', name=col))

fig.update_layout(
        title='Net positions',
        xaxis_title='Date',
        yaxis_title='Contratcs',
        template='plotly_dark',
        height=600,
        width=1500,
    )

fig.write_html("plots/net_positions.html")
fig.show()

## Plotting all charts


In [92]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: 1 column, 3 rows, shared X-axis
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing= 0.1, subplot_titles=('COT Index', 'Net positions', 'OI'))

# Add each series to its own subplot
fig.add_trace(go.Scatter(x=asset_df.index, y=commercials_df['Index'], name='COT Index'), row=1, col=1)
for col in net_positions_df.columns:
    fig.add_trace(go.Scatter(x=asset_df.index, y=net_positions_df[col], name=col), row=2, col=1)
fig.add_trace(go.Scatter(x=asset_df.index, y=oi_df['OI'], name='OI'), row=3, col=1)

fig.add_shape(
    type='rect',
    xref='paper',  # spans the full width of the plot
    yref='y1',  # y-axis of subplot in row 1
    x0=0,
    x1=1,
    y0=80,              # Adjust based on your y-axis range
    y1=100,
    fillcolor='green',
    opacity=0.2,
    line_width=0,
    layer='below'
)

fig.add_shape(
    type='rect',
    xref='paper',  # spans the full width of the plot
    yref='y1',  # y-axis of subplot in row 1
    x0=0,
    x1=1,
    y0=0,              # Adjust based on your y-axis range
    y1=20,
    fillcolor='red',
    opacity=0.2,
    line_width=0,
    layer='below'
)


# Update layout
fig.update_layout(
    height=900,
    width=1450,
    title_text='Multiple Time Series with Shared X-Axis',
    template='plotly_dark',
    hovermode='x unified',  # or 'x unified' or 'closest'
)
fig.update_xaxes(title_text='Date', row=3, col=1)  # Only show x-axis label on bottom chart
fig.update_yaxes(title_text='CONTRACT')

# Save as responsive HTML
fig.write_html("plots/cot_analysis.html", auto_open=False, full_html=True, include_plotlyjs='cdn', config={'responsive': True})
fig.show()